In [0]:
%run ./Includes

In [0]:
connectionString = "Endpoint=sb://cust-dev-eventhub-ns.servicebus.windows.net/;SharedAccessKeyName=sender;SharedAccessKey=Kappq9Y/s+FjwEePbtMnINsv7fKHhjocn+AEhPD0yIc=;EntityPath=shipments"

ehConf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
}

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

Shipments = df.selectExpr("cast(body as string) as json_data")
display(Shipments)

In [0]:
load_bronze_multiplex(Shipments,"shipments","/mnt/checkpoint/shipments/","/mnt/bronze/multiplexbronzetable")

In [0]:
%sql
select * from delta.`/mnt/bronze/multiplexbronzetable` where source='shipments';